In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

%load_ext autoreload
%autoreload 2


In [2]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

# Response to Resistance
`austintexas.gov-response_to_resistance-1.1.1` Last Update: 2020-06-10T04:54:03

_Austin Police Department response to police resistance incidents._


## Documentation Links

* [Data Dictionary](https://data.austintexas.gov/Public-Safety/GUIDE-2017-R2R/27tg-e4dt/data) A spreadsheet with field definitions, for 2017
* [R2R Datasets](https://data.austintexas.gov/browse?q=Response+To+Resistence&sortBy=relevance&page=1) A search on the Austin open data site for R2R datasets.

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [r2r_2015](data/r2r_2015.csv)**. Response to Resistance, 2015
* ** [r2r_2016](data/r2r_2016.csv)**. Response to Resistance, 2016
* ** [r2r_2017](data/r2r_2017.csv)**. Response to Resistance, 2017
* ** [r2r_2018](data/r2r_2018.csv)**. Response to Resistance, 2018
* ** [r2r_2015_2018](data/r2r_2015_2018.csv)**. Combined R2R files, 2015 to 2018

In [3]:
df = pkg.resource('r2r_2015').dataframe()
df.head().T

,0,1,2,3,4
rin,154619,154620,155009,154764,154859
primary_key,201510652,201510652,201510652,201520457,201520457
date_occurred,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-02 00:00:00,2015-01-02 00:00:00
time_occurred,0220,0225,,0943,0943
location,E 6TH ST / TRINITY ST,E 6TH ST / TRINITY ST,E 6TH ST / TRINITY ST,1900 FARO DR,1900 FARO DR
area_command,GE,GE,GE,HE,HE
nature_of_contact,VIEWED OFFENSE,VIEWED OFFENSE,VIEWED OFFENSE,DISPATCHED CALL,DISPATCHED CALL
reason_desc,NECESSARY TO EFFECT ARREST / DETENTION,NECESSARY TO EFFECT ARREST / DETENTION,NECESSARY TO EFFECT ARREST / DETENTION,TO RESTRAIN FOR SUBJECTS SAFETY,"IN CUSTODY, MAINTAINING CONTROL"
r2r_level,3,3,3,3,3
master_subject_id,459228703: 201510652,459228703: 201510652,459228703: 201510652,356649076: 201520457,356649076: 201520457


In [4]:
doc = pkg

from itertools import zip_longest

headers = []

# This will iterate through columns in roughtly the same time from 
# each dataset. The prevents a case where, for insance, the last dataset has a new column
# in an early group ( say, 5th column ) but it will end up at the end of the combined columns. 
# Whith this technique, it will end up near the 5th position. 

columns = [list(c['header'] for c in r.columns()) for r in doc.resources() if r.name != 'r2r_2015_2018']

for col_headers in columns:

    for c in col_headers:
        if c not in headers:
            headers.append(c)
            
print(headers)
        

['rin', 'primary_key', 'date_occurred', 'time_occurred', 'location', 'area_command', 'nature_of_contact', 'reason_desc', 'r2r_level', 'master_subject_id', 'subject_sex', 'subject_race', 'subject_ethnicity', 'subject_conduct_desc', 'subject_resistance', 'weapon_used_1', 'weapon_used_2', 'weapon_used_3', 'weapon_used_4', 'weapon_used_5', 'number_shots', 'subject_effects', 'effect_on_officer', 'officer_organization_desc', 'officer_commission_date', 'officer_yrs_of_service', 'x_coordinate', 'y_coordinate', 'council_district', 'census_tract', 'county', 'location_1', 'zip', 'county_description']


In [5]:

def update_columns(df, headers):
    
    t = pd.DataFrame()
    
    for h in headers:
        if h in df.columns:
            t[h] = df[h]
        else:
            t[h] = None
            
frames = [] 

for r in doc.resources():
    if r.name == 'r2r_2015_2018':
        continue
            
    df = r.dataframe()
    
    
    update_columns(df, headers)
    
    frames.append(df)
    
    
df = pd.concat(frames, sort=False)



In [6]:
# Break up the subject_resistance string in to parts, and make a dummy variable for each one. 

def xform(v):
    """Make replacements on an individual code"""
    return v.strip().lower()\
                .replace('expired code-','')\
                .replace('/','_')\
                .replace(' ','_')\
                .replace('nan','none')\
                .replace('-','none')
                
    
def expand_codes(v):
    """Expand the codes in a subject_resistance value into a list"""
    return list(xform(e) for e in v.split(';'))
    
orig_sr = list(sorted(set(e.strip() for  r in list(df.subject_resistance.unique()) for e in str(r).split(';') )))

dummy_var_names = set( xform(e) for e in orig_sr)

for dmv in dummy_var_names:
    print(f"Add dummy var 'sr_{dmv}' ")
    df['sr_'+dmv] = df.subject_resistance.fillna('').apply(lambda v: True if dmv in expand_codes(v) else False)
    


Add dummy var 'sr_none' 
Add dummy var 'sr_empty_hand_defensive_resistance' 
Add dummy var 'sr_aggressive_resistance' 
Add dummy var 'sr_evading_fleeing_motor_vehicle' 
Add dummy var 'sr_verbal_resistance_aggression' 
Add dummy var 'sr_deadly_resistance' 
Add dummy var 'sr_not_resistant' 
Add dummy var 'sr_other' 
Add dummy var 'sr_passive_resistance' 
Add dummy var 'sr_empty_hand_active_aggression' 
Add dummy var 'sr_defensive_resistance' 
Add dummy var 'sr_edged_weapon' 
Add dummy var 'sr_firearm' 
Add dummy var 'sr_preparatory_resistance' 


In [10]:
df.sr_edged_weapon.sum()
    

95

In [8]:
sr_cols = [c for c in df.columns if c.startswith('sr_')]
t = df[ ['subject_resistance']+sr_cols]
t[t.subject_resistance.fillna('').str.contains('expired')].T

,0,1,2,3,4,5,6,7,8,9,...,3282,3283,3284,3285,3286,3287,3288,3289,3290,3291
subject_resistance,expired code-evading/fleeing motor vehicle,expired code-other,expired code-verbal resistance/aggression; exp...,expired code-empty hand active aggression; exp...,expired code-verbal resistance/aggression; exp...,expired code-verbal resistance/aggression; exp...,expired code-empty hand active aggression; exp...,expired code-empty hand defensive resistance,expired code-verbal resistance/aggression; exp...,expired code-verbal resistance/aggression; exp...,...,expired code-empty hand active aggression,expired code-empty hand defensive resistance,expired code-verbal resistance/aggression; exp...,expired code-empty hand defensive resistance,expired code-verbal resistance/aggression; exp...,expired code-empty hand defensive resistance,expired code-empty hand defensive resistance,expired code-verbal resistance/aggression; exp...,expired code-verbal resistance/aggression; exp...,expired code-empty hand defensive resistance
sr_none,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
sr_empty_hand_defensive_resistance,False,False,True,False,False,True,True,True,True,True,...,False,True,True,True,True,True,True,True,True,True
sr_aggressive_resistance,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
sr_evading_fleeing_motor_vehicle,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
sr_verbal_resistance_aggression,False,False,True,False,True,True,False,False,True,True,...,False,False,True,False,True,False,False,True,True,False
sr_deadly_resistance,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
sr_not_resistant,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
sr_other,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
sr_passive_resistance,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False


In [9]:
df[df.subject_resistance.fillna('').str.contains('expired')].subject_resistance.unique()

array(['expired code-evading/fleeing motor vehicle', 'expired code-other',
       'expired code-verbal resistance/aggression; expired code-empty hand active aggression; expired code-empty hand defensive resistance',
       'expired code-empty hand active aggression; expired code-other',
       'expired code-verbal resistance/aggression; expired code-empty hand active aggression',
       'expired code-verbal resistance/aggression; expired code-empty hand defensive resistance',
       'expired code-empty hand active aggression; expired code-empty hand defensive resistance',
       'expired code-empty hand defensive resistance',
       'expired code-verbal resistance/aggression; expired code-passive resistance; expired code-empty hand defensive resistance',
       'expired code-empty hand active aggression; expired code-empty hand defensive resistance; expired code-evading/fleeing motor vehicle',
       'expired code-empty hand defensive resistance; expired code-evading/fleeing motor vehi